In [ ]:
import h5py
import numpy as np
from scipy.signal import butter, filtfilt, iirnotch
import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    b, a = butter(order, [lowcut/ nyq, highcut/ nyq], btype='band', analog=False)
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def notch(notch_freq, samp_freq, quality_factor=30):
    b, a = iirnotch(notch_freq, quality_factor, samp_freq)
    return b, a

def notch_filter(data, notch_fs, fs, q=30):
    b, a = notch(notch_fs, fs, q)
    y = filtfilt(b, a, data)
    return y

def filt_GRID(data, lowcut=20, highcut=500, fs=4000, order=3, notch_fs = 50, notch_q = 30):
    filt_out = np.zeros_like(data)
    for i in range(data.shape[0]):
        filt_out[i,:] = notch_filter(butter_bandpass_filter(data[i,:], lowcut, highcut, fs, order=order), notch_fs, fs, notch_q)
    return filt_out


In [ ]:
def loadmat(filename):
    file = h5py.File(filename, 'r')
    return np.asarray(file['grid_crds']), np.asarray(file['out_mat'])

if1_grid_crds, if1_out_mat = loadmat('./increasing force 1.mat')
if2_grid_crds, if2_out_mat = loadmat('./increasing force 2.mat')
sf1_grid_crds, sf1_out_mat = loadmat('./steady force 1.mat')

In [ ]:
print(if1_out_mat.shape)

In [ ]:
for i in range(if1_out_mat.shape[1]):
    plt.plot(if1_out_mat[:, i])
plt.show()

In [ ]:
filtered_if1_out_mat = filt_GRID(if1_out_mat)

In [ ]:
for i in range(filtered_if1_out_mat.shape[1]):
    plt.plot(filtered_if1_out_mat[:, i])
plt.show()

In [ ]:
fastica = FastICA(n_components=filtered_if1_out_mat.shape[1])
fastica_model = fastica.fit(filtered_if1_out_mat)

In [ ]:
IC = fastica_model.transform(filtered_if1_out_mat)
print(IC)

In [ ]:
for i in range(IC.shape[1]):
    plt.plot(IC[:, i])
plt.show()